# Libraries and Tools

In [1]:
from utils import *
from student_utils import *
import gurobipy as grb
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Graph Class

In [2]:
"""
A class that performs data preprocessing to gather important data and to provide useful functions
"""
class Graph:
    def __init__(self, input_file):
        parsed_data = data_parser(read_file(input_file))
        self.data = {}
        self.input_file = input_file
        self.number_of_locations = parsed_data[0] 
        self.number_of_houses = parsed_data[1] 
        self.list_of_locations = parsed_data[2]  
        self.list_of_houses = parsed_data[3] 
        self.starting_location = parsed_data[4] 
        self.adjacency_matrix = parsed_data[5] 
        self.G, message = adjacency_matrix_to_graph(self.adjacency_matrix)
        if message:
            print(message)
        else:
            print("Successful creation of Graph instance")
        
    """
    Returns a NumPy matrix of shortest path distances between every pair of nodes using the Floyd-Warshall algorithm. 
    If there is no path between to nodes the corresponding matrix entry will be Inf.
    """
    def get_fw_matrix(self):
        return nx.floyd_warshall_numpy(self.G)

In [3]:
graph = Graph("inputs/84_50.in")

Successful creation of Graph instance


# Integer Linear Programming

In [4]:
model = grb.Model()

Academic license - for non-commercial use only


In [5]:
"""
Arrangement of Drop-Offs Matrix
"""
ARRANGEMENTS = []
for r in range(graph.number_of_locations):
    ROW = []
    for c in range(graph.number_of_houses + 2):
        matrix_element_name = "A_" + str(r) + "_" + str(c)
        ROW.append(model.addVar(vtype=grb.GRB.BINARY, name=matrix_element_name))
        model.update()
    ARRANGEMENTS.append(ROW)
ARRANGEMENTS = np.array(ARRANGEMENTS)

In [6]:
"""
Arrangement of Drop-Offs Constraints
"""
# Check that we start at Soda
model.addConstr(ARRANGEMENTS[0][0] == 1)
# Check that we end at Soda
model.addConstr(ARRANGEMENTS[0][graph.number_of_houses + 1] == 1)
# Check that each column of ARRANGEMENTS sums up to 1
for c in range(len(ARRANGEMENTS[0])):
    model.addConstr(grb.quicksum(ARRANGEMENTS[:, c]) == 1)

In [7]:
"""
TA Walking Matrix
"""
WALKING = []
for r in range(graph.number_of_locations):
    ROW = []
    for c in range(graph.number_of_locations):
        matrix_element_name = "W_" + str(r) + "_" + str(c)
        ROW.append(model.addVar(vtype=grb.GRB.BINARY, name=matrix_element_name))
        model.update()
    WALKING.append(ROW)
WALKING = np.array(WALKING)

In [ ]:
"""
TA Walking Constraints
"""


In [ ]:
"""
Driving Cost Function
"""
DISTANCES = graph.get_fw_matrix()
driving_cost_function = []
for c in range(graph.number_of_houses + 1):
    summation = []
    for i in range(graph.number_of_locations):
        for j in range(graph.number_of_locations):
            summation.append(
                grb.QuadExpr(ARRANGEMENTS[i][c] * DISTANCES.item((i, j)) * ARRANGEMENTS[j][c + 1])
            )
            model.update()
    driving_cost_function.append(summation)

In [ ]:
"""
Walking Cost Function
"""
walking_cost_function = []
for r in range(graph.number_of_locations):
    for c in range(graph.number_of_locations):
        walking_cost_function.append(grb.LinExpr(DISTANCES.item((i, j))))
        model.update()